In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import sys

import torch
import fastai
from fastai.imports import *
from fastai.vision import *
from fastai.vision.all import *
from torchvision.models import vgg16_bn
from torchvision import datasets, transforms

import PIL
import numpy as np 
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import image
from matplotlib.pyplot import imshow
from sklearn import preprocessing

In [ ]:
## Defining Directories
work_dir = Path('/kaggle/working/')
path = Path('../input/hotel-id-2021-fgvc8')
train = path/ '/train_images'
test =  path/'/test_images'
sample_sub = path/'sample_submission.csv'
labels = path/'train.csv'

## Reading csv
train_data = pd.read_csv(labels)
df_sample = pd.read_csv(sample_sub)

train_data.head()
train_data

In [ ]:
train

In [ ]:
train_data.columns
len_data = len(train_data['hotel_id'].value_counts())

print("data Length:", len_data)
print("Hotel Chains:",train_data['chain'].unique())
chain_len = len(train_data['chain'].unique())-1
chain_len


In [ ]:
# Hotels per Chain 
hotel_per_chain={}
chains = train_data['chain'].unique()
for chain_id in chains:
    hotel_per_chain[chain_id] = train_data[train_data['chain']==chain_id]['hotel_id'].nunique()

#plot
bar = plt.bar(x=hotel_per_chain.keys(),height=hotel_per_chain.values())
plt.xlabel("Chain ID")
plt.ylabel("Count")
plt.title("Hotels per Chain")

In [ ]:
## Images per Hotel
image_per_hotel=[]
hotels_id = train_data['hotel_id'].unique()
for hotel_id in hotels_id:
    image_per_hotel.append(train_data[train_data['hotel_id']==hotel_id]['image'].nunique())
#     print(image_per_hotel)
  
hotel_image_df = pd.DataFrame({"hotel_id":map(str,hotels_id),"image_count":image_per_hotel})
hotel_image_df.sort_values(by="image_count",ascending=False,inplace=True)  

plt.figure(figsize=[15,15])

plt.bar(x=hotel_image_df["hotel_id"],height=hotel_image_df["image_count"])
plt.xlabel("Hotel ID")
# plt.xticks(rotation=45)
plt.ylabel("Image Count")
# plt.title("Hotel and their image count (Top 50)")

In [ ]:
# label_enc = preprocessing.LabelEncoder().fit(train_data['hotel_id'])
# train_data['label'] = label_enc.transform(train_data['hotel_id'])
# train_data

In [ ]:
def data_loader(img_path):
    img_size = (224, 168)
    normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], 
                                        std=[0.5, 0.5, 0.5])
    
    transform = transforms.Compose([transforms.Resize(img_size),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    normalize
                                   ])
    
    dataset = datasets.ImageFolder(img_path, transform=transform)
    
    class_to_idx = dataset.class_to_idx 
    idx_to_class = {v: int(k) for k,v in class_to_idx.items()}
    
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False) 
    
    return dataloader, idx_to_class
    

In [ ]:
img_path = str(path)  + str(train)  
print(img_path)

In [ ]:
dataloader, idx_to_class = (data_loader(img_path))

In [ ]:
images, labels = next(iter(dataloader))

In [ ]:
images.shape

In [ ]:
img = images[0].permute(1, 2, 0)
imshow(np.asarray(img))

### Extract feature vector for an tensor representing an image


In [ ]:
def gram_matrix(x):
    n,c,h,w = x.size()
    x = x.view(n, c, -1)
    return (x @ x.transpose(1,2))/(c*h*w)

In [ ]:
def compute_features(x, feature_extractor, layer_ids, clone=False):
    features = [feature_extractor[i] for i in layer_ids]
    hooks = hook_outputs(features, detach=False)
    feature_extractor(x)
    features_x = [(o.clone() if clone else o) for o in hooks.stored]
    return features_x

In [ ]:
def compute_feature_vector(x, feature_extractor, layer_ids, clone=False):
    features_x = compute_features(x, feature_extractor, layer_ids, clone)
    # Computing the Gram Matrix for the features
    gmv = [gram_matrix(e) for e in features_x]
    # flat to get single vector
    gmv_flat_vector = torch.cat([torch.flatten(g) for g in gmv])
    return gmv_flat_vector

In [ ]:
# using a vgg16 network as feature extractor
vgg_m = vgg16_bn(True).features.cuda().eval()
vgg_m.requires_grad_(False);

In [ ]:
requires_grad(vgg_m)

In [ ]:
# blocks (i.e., layers) to use for the feature extraction

blocks = [i-1 for i,o in enumerate(children_and_parameters(vgg_m)) if isinstance(o,nn.MaxPool2d)]
blocks, [vgg_m[i] for i in blocks]

In [ ]:
images[0].unsqueeze(0).shape

In [ ]:
imagesCuda, labelsCuda = images.cuda(), labels.cuda()

In [ ]:
gmv_flat_vector =[]
for i in range(imagesCuda.shape[0]):
    vector = (compute_feature_vector(imagesCuda[i].unsqueeze(0), vgg_m, blocks))
    gmv_flat_vector.append(torch.cat([torch.flatten(g) for g in vector]))


Class

In [ ]:
vector.shape


In [ ]:
(gmv_flat_vector[0] - vector[0]).cpu().norm()

In [ ]:
gmv_flat_vector[0]

In [ ]:
len(gmv_flat_vector)
